#### Mapping Declaratively with Reflected Tables

There are _several patterns available_ which provide for producing _mapped classes_ against a _series_ of `Table` objects that were __introspected from the database__, using the _reflection process_ described at `Reflecting Database Objects`.

A _very simple way_ to __map a class to a table reflected from the database__ is to use a `declarative hybrid mapping`, passing the `Table.autoload_with` parameter to the `Table`.

In [1]:
from sqlalchemy import Column, Integer, String, Table, create_engine
from sqlalchemy.orm import declarative_base

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
Base = declarative_base()

In [4]:
class UserInline(Base):
    __table__ = Table(
        "user",
        Base.metadata,
        Column("id", Integer, primary_key=True),
        Column("name", String),
        Column("fullname", String),
        Column("nickname", String),
    )

In [5]:
Base.metadata.create_all(engine)

2022-11-01 15:20:12,814 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-01 15:20:12,816 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2022-11-01 15:20:12,816 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-01 15:20:12,818 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user")
2022-11-01 15:20:12,819 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-01 15:20:12,821 INFO sqlalchemy.engine.Engine 
CREATE TABLE user (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2022-11-01 15:20:12,823 INFO sqlalchemy.engine.Engine [no key 0.00204s] ()
2022-11-01 15:20:12,826 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
class User(Base):
    __table__ = Table(
        "user",
        Base.metadata,
        autoload_with=engine,
    )

A _variant_ on the above pattern that __scales much better__ is to use the `MetaData.reflect()` method to __reflect a full set of `Table` objects at once__, then refer to them from the `MetaData`.

In [7]:
Base.metadata.reflect(engine)

2022-11-01 15:21:22,256 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-01 15:21:22,258 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-11-01 15:21:22,259 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-01 15:21:22,263 INFO sqlalchemy.engine.Engine ROLLBACK


In [8]:
class ReflectedUser(Base):
    __table__ = Base.metadata.tables["user"]

A __major downside__ to the above approach is that the _mapped classes cannot be declared until the tables have been reflected_, which __requires__ the `database connectivity source` __to be present__ while the `application classes` are _being declared_; it's _typical_ that _classes are declared as the modules of an application are being imported_, but `database connectivity` __isn't available until the application starts__ running code so that it can _consume configuration information_ and __create an engine__. There are _currently_ `two approaches` to __working around__ this.